<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Effects Model with Genomic Data</font> 
</div>

In [1]:
using DataFrames,CSV,JWAS
using JWAS: Datasets,misc

In [2]:
?runMCMC

search: runMCMC



```
runMCMC(mme,df;Pi=0.0,estimatePi=false,chain_length=1000,starting_value=false,printout_frequency=100,missing_phenotypes=false,constraint=false,methods="conventional (no markers)",output_samples_frequency::Int64 = 0)
```

Run MCMC (marker information included or not) with sampling of variance components.

  * available **methods** include "conventional (no markers)", "BayesC0", "BayesC", "BayesCC","BayesB".
  * **missing_phenotypes**
  * **Pi** for single-trait analyses is a number; **Pi** for multi-trait analyses is a dictionary such as `Pi=Dict([1.0; 1.0]=>0.7,[1.0; 0.0]=>0.1,[0.0; 1.0]=>0.1,[0.0; 0.0]=>0.1)`,

      * if Pi (Π) is not provided in multi-trait analysis, it will be generated assuming all markers have effects on all traits.
  * save MCMC samples every **output_samples_frequency** iterations
  * **starting_value** can be provided as a vector for all location parameteres except marker effects.
  * print out the monte carlo mean in REPL with **printout_frequency**
  * **constraint**=true if constrain residual covariances between traits to be zero.


<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [7]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Source#24(::String, ::CSV.Options{Void}, ::Bool, ::Int64, ::Array{Type,1}, ::Symbol, ::Void, ::Bool, ::Bool, ::Int64, ::Int64, ::Int64, ::Bool, ::Type{T} where T) at /Users/haocheng/.julia/v0.6/CSV/src/Source.jl:61
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{CSV.Source}) at ./<missing>:0
 [4] #Source#23(::Char, ::UInt8, ::UInt8, ::String, ::String, ::Bool, ::Int64, ::Array{Type,1}, ::Symbol, ::Void, ::Void, ::UInt8, ::String, ::String, ::Bool, ::Bool, ::Int64, ::Int64, ::Int64, ::Bool, ::Type{T} where T, ::String) at /Users/haocheng/.julia/v0.6/CSV/src/Source.jl:30
 [5] (::Core.#kw#Type)(::Array{Any,1}, ::Type{CSV.Source}, ::String) at ./<missing>:0
 [6] #read#43(::Bool, ::Dict{Int64,Function}, ::Bool, ::Array{Any,1}, ::Function, ::String, ::Type{T} where T) at /Users/haocheng/.julia/v0.6/CSV/src/Source.jl:338
 [7] (::CSV.#kw##read)(::Array{Any,1}, ::CSV.#read, ::String, ::Type{T} where T) at ./<missing>:0 (repe

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multiple Traits Analyses with Marker Information</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [8]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [9]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [10]:
model1 = build_model(model_equations,R);

In [11]:
set_covariate(model1,"age");

In [12]:
add_genotypes(model1,genofile,G,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
Pi=Dict([1.0; 1.0]=>0.7,[1.0;0.0]=>0.1,[0.0,1.0]=>0.1,[0.0; 0.0]=>0.1)
out = runMCMC(model1,data,Pi=Pi,chain_length=5000,methods="BayesC",
estimatePi=true,output_samples_frequency=5);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[10.9589 0.626223; 0.626223 1.09589].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          5
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.1,[1.0, 0.0]=>0.1,[1.0, 1.0]=>0.7,[0.0, 0.0]=>0.1)
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.0

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [14]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [15]:
file1="MCMC_samples_for_marker_effects_BW.txt"
file2="MCMC_samples_for_marker_effects_CW.txt";

In [16]:
get_breeding_values(model1,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV      │ PEV     │
├─────┼────┼──────────┼─────────┤
│ 1   │ S1 │ -1.16575 │ 7.48027 │
│ 2   │ D1 │ 4.11458  │ 58.1247 │
│ 3   │ O1 │ 3.34231  │ 63.8867 │
│ 4   │ O3 │ -6.29114 │ 56.2938 │            
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV       │ PEV      │
├─────┼────┼───────────┼──────────┤
│ 1   │ S1 │ -0.218614 │ 0.427367 │
│ 2   │ D1 │ 0.829118  │ 2.94729  │
│ 3   │ O1 │ 0.700563  │ 3.5645   │
│ 4   │ O3 │ -1.31107  │ 3.48593  │

In [17]:
samples4G=get_additive_genetic_variances(model1,file1,file2);

samples4R=out["MCMC samples for residual covariance matrix"];

samples4h2=get_heritability(reformat(samples4G),reformat(samples4R));

In [18]:
writedlm("out3.txt",samples4G)

In [19]:
samples_genetic_correlation=get_correlations(reformat(samples4G));

In [20]:
writedlm("out.G.txt",reformat(reformat(samples4G)))

In [21]:
out=readdlm("out.G.txt")

4×1000 Array{Float64,2}:
  75.9576   214.199     169.557     …   4.43168  24.023     0.0176831
 -11.5904    -2.76706    -4.47821      -1.3264    4.5395   -0.185    
 -11.5904    -2.76706    -4.47821      -1.3264    4.5395   -0.185    
   2.25022    0.109326    0.134539      1.85078   4.13379   1.93546  

In [22]:
reformat(out)

1000-element Array{Array{Float64,2},1}:
 [75.9576 -11.5904; -11.5904 2.25022]      
 [214.199 -2.76706; -2.76706 0.109326]     
 [169.557 -4.47821; -4.47821 0.134539]     
 [54.4354 0.0; 0.0 0.0]                    
 [20.4455 -2.07862; -2.07862 0.285873]     
 [10.2694 0.0749113; 0.0749113 0.00117372] 
 [0.0 0.0; 0.0 0.215269]                   
 [0.931946 -0.438033; -0.438033 0.438992]  
 [20.2241 0.299041; 0.299041 0.617251]     
 [29.5567 -0.427954; -0.427954 0.465983]   
 [23.0433 0.988663; 0.988663 0.18153]      
 [2.65334 0.186062; 0.186062 0.109608]     
 [12.5931 7.2791e-18; 7.2791e-18 0.0356108]
 ⋮                                         
 [34.7272 4.46529; 4.46529 1.14531]        
 [26.0568 4.84291; 4.84291 1.65511]        
 [0.0 0.0; 0.0 0.349961]                   
 [24.7707 4.88417; 4.88417 1.2596]         
 [67.5104 7.75837; 7.75837 1.06718]        
 [98.372 20.2698; 20.2698 4.60033]         
 [1.27719 -0.0841033; -0.0841033 0.143608] 
 [10.5883 0.300446; 0.300446 0.54207

In [23]:
out10=reformat(samples4h2)

2×1000 Array{Float64,2}:
 0.320643  0.736431   0.73948    …  0.0144466  0.235933  0.000176245
 0.180494  0.0231696  0.0146801     0.0661075  0.571031  0.255833   

In [24]:
reformat(out,2)

1000-element Array{Array{Float64,2},1}:
 [75.9576 -11.5904; -11.5904 2.25022]      
 [214.199 -2.76706; -2.76706 0.109326]     
 [169.557 -4.47821; -4.47821 0.134539]     
 [54.4354 0.0; 0.0 0.0]                    
 [20.4455 -2.07862; -2.07862 0.285873]     
 [10.2694 0.0749113; 0.0749113 0.00117372] 
 [0.0 0.0; 0.0 0.215269]                   
 [0.931946 -0.438033; -0.438033 0.438992]  
 [20.2241 0.299041; 0.299041 0.617251]     
 [29.5567 -0.427954; -0.427954 0.465983]   
 [23.0433 0.988663; 0.988663 0.18153]      
 [2.65334 0.186062; 0.186062 0.109608]     
 [12.5931 7.2791e-18; 7.2791e-18 0.0356108]
 ⋮                                         
 [34.7272 4.46529; 4.46529 1.14531]        
 [26.0568 4.84291; 4.84291 1.65511]        
 [0.0 0.0; 0.0 0.349961]                   
 [24.7707 4.88417; 4.88417 1.2596]         
 [67.5104 7.75837; 7.75837 1.06718]        
 [98.372 20.2698; 20.2698 4.60033]         
 [1.27719 -0.0841033; -0.0841033 0.143608] 
 [10.5883 0.300446; 0.300446 0.54207

In [25]:
#genetic correlation between trait 1 and trait 2
report(reformat(samples4G),index=[1,2]);

Summary Stats:
Mean:           15.336644
Minimum:        -14.407621
1st Quartile:   0.000000
Median:         1.024724
3rd Quartile:   27.529307
Maximum:        127.681809
Length:         1000
Type:           Float64
nothing


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. Multiple Traits Analyses with Marker Effects and Polygenic Effects</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [26]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";
model2          = build_model(model_equations,R);

set_covariate(model2,"age");

get pedigree information from a file

In [27]:
ped=get_pedigree(pedfile);

Finished!


In [28]:
GA = G*0.1
set_random(model2,"Animal",ped,GA)

In [29]:
GM = G*0.9
add_genotypes(model2,genofile,GM,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [30]:
out2=runMCMC(model2,data,chain_length=5000,methods="BayesC");

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[7.89041 0.394521; 0.394521 0.789041].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.0,[1.0, 0.0]=>0.0,[1.0, 1.0]=>1.0,[0.0, 0.0]=>0.0)
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [31]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"

In [32]:
out2["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
 2.08137   0.135832
 0.135832  0.190496